In [4]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))

import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry


# soup = BeautifulSoup(html, 'html.parser')
# items = soup.find_all('option')

In [38]:
from urllib.parse import urljoin
import re
result_list = []
pattern = re.compile(r"PU\('(.*?)'\)")
for i in range(1, 21):
    response = session.get(f'https://fieo.org/searchItcHcCode_fieo.php?stype=Like&searchStringProducts={("0"+str(i))[-2:]}')
    soup = BeautifulSoup(response.text, 'html.parser')

    table_list = soup.find_all('table')
    table = table_list[13]
    for tr in table.find_all('tr')[1:]:
        td_list = tr.find_all('td')
        
        match = pattern.search(td_list[4].find('a').get('href')).group(1)  # 첫 번째 캡처 그룹의 내용

        page_response = session.get(urljoin('https://fieo.org/', match))
        page_soup = BeautifulSoup(page_response.text, 'html.parser')

        page_table_list = page_soup.find_all('table')
        sku_list = []
        for tr in page_table_list[6].find_all('tr')[1:]:
            td_tags = tr.find_all('td')
            sku_list.append(td_tags[0].text.strip())

        item = {
            'name': page_table_list[5].find('tr').find('td').text.strip(),
            'sku': ', '.join(sku_list)
        }
        result_list.append(item)
    df = pd.DataFrame(result_list)
    df.to_csv('./output/fieo_20240213.csv', index=False, quoting=1)
    